### Importation des bibliothèques nécessaires

In [ ]:
import pandas as pd
import random
import numpy as np
from aizynthfinder.chem import Molecule, Reaction, MoleculeException
from aizynthfinder.training.utils import (
    Config,
    create_reactants_molecules,
    reverse_template,
    reaction_hash,
    reactants_to_fingerprint,
)
from aizynthfinder.utils.models import CUSTOM_OBJECTS, load_keras_model

### Récupération des bases de données

In [ ]:
data_reactions = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-reactions.json")
data_templates = pd.read_json("C:/Users/Yassine/Desktop/aizynth/uspto-templates.json")

### Création des fonctions dont on va avoir besoin

In [ ]:
def get_random_row(lib):
    '''
    Renvoie une ligne aléatoire dans un DataFrame Panda
    '''
    
    r = random.randint(0,lib.shape[0]-1)
    return [lib.iloc[r]]

In [ ]:
def random_sample_lib(data_templates, data_reactions):
    '''
    Renvoie une liste de nouvelles "fausses" réactions en applicant des templates pris aléatoirement 
    dans la base correspondante. 
    
    '''
    to_return = []
    
    
    for val, row in data_reactions.iterrows():   #On parcours l'ensemble des réactions
        
        if (type(row.reactants) == str):   #On vérifie que les réactifs sont bien présent (les bases de données 
                                            #sont parfois incomplètes)
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
            
        try:   #On récupère le produit au format Molecule pour pouvoir la comparer plus loin
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
            
        #On tente de créer un nouveau produit de manière random
        new_product = None
        for template_row in get_random_row(data_templates): #On prend un template au hasard

            if (type(template_row["reaction_smarts"]) != str): #On vérifie que le template est bon
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"]) #On renverse le template car la base de donnée 
                                                                            #contient des templates de retro-reactions
            
            
            try:   #On tente d'appliquer le template tiré aux réactifs
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
                
                
            if new_product.basic_compare(ref_mol):  #On verifie que l'on n'a pas créé la même réaction.
                continue
            break  # Si on est arrivé ici, c'est qu'on a réussi à créer une "fausse réaction"

        if not new_product: #On vérifie qu'on n'a pas créé un produit vide
            continue
        
        to_return += [val,new_product] #On ajoute la réaction créée
    return to_return

In [ ]:
##A PRIORI PAS BON MAIS A CHECKER

def _sample_lib(library, lib2):
    to_return = []
    for val, row in lib2.iterrows():
        if (type(row.reactants) == str):
            mols = create_reactants_molecules(row.reactants)
        else:
            continue
        try:
            ref_mol = Molecule(smiles=row.products, sanitize=True)
        except MoleculeException:
            continue
            
        new_product = None
        for idx,template_row in library.iterrows(): #SAMPLER 
            #if row.template_hash == template_row.template_hash: #Check
            #    continue
            #smarts_fwd = reverse_template(template_row.retro_template) #Check
            if (idx != 15):
                continue
            if (type(template_row["reaction_smarts"]) != str):
                continue
            smarts_fwd = reverse_template(template_row["reaction_smarts"])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            if new_product.basic_compare(ref_mol):
                continue
            break  # If we have reached here, we have found a match that fits all criteria

        if not new_product:
            continue
        
        to_return += [val,new_product]
    return to_return

In [ ]:
#On récupère les fausses réactions crées
y = random_sample_lib(data_templates, data_reactions)
z = y[1::2]

In [ ]:
#On réécrit tout sous forme de réactions smiles
n = len(z)
arr_to_dat = np.empty(n, dtype="object")
for x in range(n):
    arr_to_dat[x] = data_reactions.iloc[y[2*x]].reactants + ">>" + z[x].smiles

In [ ]:
false_reactions = pd.DataFrame(data=arr_to_dat)
#On enregistre ensuite au format csv
false_reactions.to_csv("Nom_désiré.csv")